In [1]:
# import sys
# import os

In [2]:
# sys.path.append('/home/super/waterffle/playlist')

In [3]:
# print(sys.path)

In [4]:
# from preprocessing import load_poly_encoder_dataset
# now_mfcc_list, next_mfcc_list, label_list = load_poly_encoder_dataset(2800)

In [5]:
# bsz * 10 개만 할당
# now_mfcc_list = torch.tensor(now_mfcc_list[:320])
# next_mfcc_list = torch.tensor(next_mfcc_list[:320])
# label_list = torch.tensor(label_list[:320])

In [6]:
# gpu 메모리 일부로 제한
# torch.cuda.set_per_process_memory_fraction(0.5,1)

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [8]:
from preprocessing import load_poly_encoder_dataset

In [9]:
# device_num = 1

In [10]:
last_pad_length = 2800

In [11]:
cs, ns, labels = load_poly_encoder_dataset(last_pad_length)

====== Get MFCC with padding and crop======


100%|██████████████████████████████████████████████████████████| 32000/32000 [00:22<00:00, 1393.06it/s]


In [12]:
# bsz * 10 개만 할당
part = 3200
cs = torch.tensor(cs[:part])
ns = torch.tensor(ns[:part])
labels = torch.tensor(labels[:part])

# cs = torch.tensor(cs)
# ns = torch.tensor(ns)
# labels = torch.tensor(labels)

In [16]:
cs = cs.view(-1, last_pad_length, 40)
ns = ns.view(-1, last_pad_length, 40)

In [17]:
cs.size(), ns.size()

(torch.Size([3200, 2800, 40]), torch.Size([3200, 2800, 40]))

In [15]:
cs[0].size()

torch.Size([1, 2800, 40])

-----------------------------

In [17]:
import logging

logger = logging.getLogger(__name__)

In [18]:
def batchify(data, bsz):
    cs_rep, ns_rep, labels = data
    length = cs_rep.size()[0]
    for i in range(0, length, bsz):
        yield cs_rep[i:i+bsz], ns_rep[i:i+bsz], labels[i:i+bsz]

In [19]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math
from typing import Any, Dict, Optional, Tuple, Union
from __future__ import absolute_import, division, print_function, unicode_literals

from collections.abc import Iterable
from itertools import repeat

"""Near infinity, useful as a large penalty for scoring when inf is bad."""
NEAR_INF = 1e20
NEAR_INF_FP16 = 65504

def _pair(v):
    if isinstance(v, Iterable):
        assert len(v) == 2, "len(v) != 2"
        return v
    return tuple(repeat(v, 2))


class BasicAttention(nn.Module):
    """
    Implements simple/classical attention.
    """

    def __init__(
        self,
        dim: int = 1,
        attn: str = 'cosine',
        residual: bool = False,
        get_weights: bool = True,
    ):
        super().__init__()
        if attn == 'cosine':
            self.cosine = nn.CosineSimilarity(dim=dim)
        self.attn = attn
        self.dim = dim
        self.get_weights = get_weights
        self.residual = residual

    def forward(
        self,
        xs: torch.Tensor,
        ys: torch.Tensor,
        mask_ys: Optional[torch.Tensor] = None,
        values: Optional[torch.Tensor] = None,
    ) -> Union[torch.Tensor, Tuple[torch.Tensor, torch.Tensor]]:
        """
        Compute attention.
        Attend over ys with query xs to obtain weights, then apply weights to
        values (ys if yalues is None)
        Args:
            xs: B x query_len x dim (queries)
            ys: B x key_len x dim (keys)
            mask_ys: B x key_len (mask)
            values: B x value_len x dim (values); if None, default to ys
        """
        bsz = xs.size(0)
        y_len = ys.size(1)
        x_len = xs.size(1)
        if self.attn == 'cosine':
            l1 = self.cosine(xs, ys).unsqueeze(self.dim - 1)
        else:
            l1 = torch.bmm(xs, ys.transpose(1, 2))
            if self.attn == 'sqrt':
                d_k = ys.size(-1)
                l1 = l1 / math.sqrt(d_k)
        if mask_ys is not None:
            attn_mask = (mask_ys == 0).view(bsz, 1, y_len)
            attn_mask = attn_mask.repeat(1, x_len, 1)
            l1.masked_fill_(attn_mask, neginf(l1.dtype))
        l2 = F.softmax(l1, dim=self.dim, dtype=torch.float).type_as(l1)
        if values is None:
            values = ys
        lhs_emb = torch.bmm(l2, values)

        # # add back the query
        if self.residual:
            lhs_emb = lhs_emb.add(xs)
        
        res = lhs_emb.squeeze(self.dim - 1)
        if self.get_weights:
            return res, l2
        else:
            return res
        

class PolyBasicAttention(BasicAttention):
    """
    Override basic attention to account for edge case for polyencoder.
    """

    def __init__(self, poly_type, n_codes, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.poly_type = poly_type
        self.n_codes = n_codes

    def forward(self, *args, **kwargs):
        """
        Forward pass.
        Account for accidental dimensionality reduction when num_codes is 1 and the
        polyencoder type is 'codes'
        """
        lhs_emb = super().forward(*args, **kwargs)
        if self.poly_type == 'codes' and self.n_codes == 1 and len(lhs_emb.shape) == 2:
            lhs_emb = lhs_emb.unsqueeze(self.dim - 1)
        return lhs_emb
    

def neginf(dtype: torch.dtype) -> float:
    """
    Return a representable finite number near -inf for a dtype.
    """
    if dtype is torch.float16:
        return -NEAR_INF_FP16
    else:
        return -NEAR_INF
    

class AudioEncoder(nn.Module):
    '''
    code : https://github.com/pytorch/fairseq/blob/c36294ea4fd35eac757f417de9668b32c57d4b3d/examples/speech_recognition/models/vggtransformer.py#L271
    '''
    def __init__(self,
                 poly_n_codes, # m, the number of global contxt features
                 poly_attention_num_heads,
                 num_att_layers,
                 codes_attention_num_heads,
                 embed_dim, 
                 layer_norm=False,
                 dropout=0.1,
                 reduction_type='first' # first, avg, max
                ):
        super(AudioEncoder, self).__init__()
        
        self.n_codes = poly_n_codes
        self.attention_num_heads = poly_attention_num_heads
        self.codes_attention_num_heads = codes_attention_num_heads

        # the codes
        codes = torch.empty(self.n_codes, embed_dim)
        codes = torch.nn.init.uniform_(codes)
        self.codes = torch.nn.Parameter(codes)

        # attention for the codes
        self.code_attention = PolyBasicAttention(poly_type='codes', n_codes=self.n_codes, dim=2, attn='basic', get_weights=False)

        # The final attention (the one that takes the candidate as key)
        self.attention = nn.MultiheadAttention(embed_dim=embed_dim, 
                                               num_heads=self.attention_num_heads, 
                                               dropout=0.0)
#         self.attention = PolyBasicAttention(poly_type='codes', n_codes=self.n_codes, dim=2, attn='basic', get_weights=False)

        
        self.att_encoder_block = nn.ModuleList()
        
        # SelfAttentionEncoder * num_att_layers
        for i in range(num_att_layers):
            self.att_encoder_block.append(
                torch.nn.TransformerEncoderLayer(d_model=embed_dim, 
                                                 nhead=self.attention_num_heads, 
                                                 dim_feedforward=embed_dim*4, 
                                                 dropout=dropout, 
                                                 activation='gelu')
            )
                            
        self.att_encoder_block = nn.Sequential(*self.att_encoder_block)

            
    def attend(self, attention_layer, queries, keys, values, mask):
        """
        Apply attention.
        :param attention_layer:
            nn.Module attention layer to use for the attention
        :param queries:
            the queries for attention
        :param keys:
            the keys for attention
        :param values:
            the values for attention
        :param mask:
            mask for the attention keys
        :return:
            the result of applying attention to the values, with weights computed
            wrt to the queries and keys.
        """
        if keys is None:
            keys = values
        if isinstance(attention_layer, PolyBasicAttention):
            return attention_layer(queries, keys, values=values, mask_ys=mask)
        elif isinstance(attention_layer, nn.MultiheadAttention):
            return attention_layer(query=queries, key=keys, value=values, attn_mask=mask)[0]

        else:
            raise Exception('Unrecognized type of attention')

    def encode(self, x_raw):
        # x_raw = [current song, next song]
        # next: candidate
        cs, ns, label = x_raw

        # padded tensor
        # B, C, T, F
        bsz, in_channels, max_seq_len, _ = ns.size()

        # cand mfcc를 conv encoder를 거친 emb
        cand_emb = self.conv_encoder_block(ns)

        bsz, _, output_seq_len, _ = cand_emb.size()

        # (B, C, T, F) -> (B, T, C, F) -> (B, T, C*F)
        cand_emb = cand_emb.transpose(1, 2)
        cand_emb = cand_emb.contiguous().view(bsz, output_seq_len, -1)

        # transformer encoder
        cand_emb = self.att_encoder_block(cand_emb)

        # reduction : first, avg, max
        if self.reduction_type=='first':
            cand_emb = cand_emb[:,0,:]
        elif self.reduction_type == 'avg':
            cand_emb = torch.mean(cand_emb, dim=1)
        elif self.reduction_type == 'max':
            cand_emb = torch.max(cand_emb, dim=1).values
        else:
            raise KeyError('Not Registered reduction_type. Capable options : first, avg, and max')
        cand_emb = cand_emb.view(cand_emb.size()[0], 1, cand_emb.size()[1])

        # ctxt mfcc를 conv encoder를 거친 emb
        ctxt_out = self.conv_encoder_block(cs)
        b, c, t, f = ctxt_out.size()

        # (B, C, T, F) -> (B, T, C, F) -> (B, T, C*F)
        ctxt_out = ctxt_out.transpose(1, 2)
        ctxt_out = ctxt_out.contiguous().view(b, t, -1)

        # transformer encoder
        ctxt_out = self.att_encoder_block(ctxt_out)

        return ctxt_out, cand_emb, label
                
    def forward(self, 
                x_raw=None, 
                x_rep=None
               ):
        '''
        encoding 과정과 그 이후 과정을 나눈 이유는 먼저 계속 사용하는 피쳐를 처리해놓고 재사용하는 과정을 거치기 위해서이다.
        '''
        if x_raw is not None:
            
            return self.encode(x_raw)
        elif x_rep is not None:
            ctxt_out, cand_emb = x_rep

            # m개 만큼 context code를 반복
            # ctxt_out 값과 code를 내적한 값들의 softmax한 벡터 (w_1,...,w_m)를 이전 레이어 결과값(ctxt_out)과 곱해서 합한다.
            # 이 값이 m개의 global context features
            bsz = cand_emb.size(0)
            global_ctxts = self.attend(attention_layer=self.code_attention , 
                                       queries=self.codes.repeat(bsz, 1, 1), 
                                       keys=ctxt_out,
                                       values=ctxt_out, 
                                       mask=None) 
        
            # torch.nn.MultiheadAttention은 (seq_len, bsz, embed_dim) 으로 입력값을 받아서 수정
            global_ctxts = global_ctxts.transpose(0,1)
            
            cand_emb = cand_emb.transpose(0,1)
            
            # m개의 global context features를 cand_emb와 내적한 값을 softmax한 벡터를 (w_1,...,w_m)라 할 때, 이 가중치 값과 global contxt features를 곱해서 합한다.
            # 이 값이 최종 ctxt_emb
            ctxt_emb = self.attend(attention_layer=self.attention ,
                                   queries=cand_emb, 
                                   keys=global_ctxts,
                                   values=global_ctxts,
                                   mask=None)
#             ctxt_emb = ctxt_emb.transpose(0, 1)

            # score: cand_emb와 ctxt_emb 간 cosine similarity값 (반환값)
            scores = torch.sum(ctxt_emb * cand_emb, -1)
            scores = scores.view(1, -1)
            return scores
        else:
            raise Exception('Unsupported operation')
    
    def infer_conv_output_dim(self, in_channels, input_dim):
        sample_seq_len = 200
        sample_bsz = 10
        x = torch.randn(sample_bsz, in_channels, sample_seq_len, input_dim)
        for i, _ in enumerate(self.conv_encoder_block):
            x = self.conv_encoder_block[i](x)
        # (B, C, T, F) -> (B, T, C, F) -> (B, T, C*F)
        x = x.transpose(1, 2)
        mb, seq = x.size()[:2]
        return x.contiguous().view(mb, seq, -1).size(-1)


In [20]:
model = AudioEncoder(
                     poly_n_codes=64, # m, the number of global contxt features
                     poly_attention_num_heads=8, 
                     codes_attention_num_heads=2,
                     num_att_layers=6,
                     embed_dim=512, 
                     input_feat_per_channel=40, # feature vector dimension
                     num_conv_block=2,
                     num_conv_layers=2, 
                     in_channels=1, 
                     out_channels=32,
                     conv_kernel_size=3, 
                     pooling_kernel_size=2,
                     layer_norm=True,
                     reduction_type='first'
)

In [21]:
from sklearn.metrics import accuracy_score

--------------------------------------------

In [22]:
def eval_model(model, dataloader, criterion, device):
    # layer 중 dropout layer와 같이 학습 시에는 사용하는데 inference할 때는 사용하지 않는 경우를 구분해주기 위함
    model.eval()
    
    n_steps = 0
    eval_loss = 0    
    acc = 0

    # autograd engine을 끔(gradient 계산하지 않음) --> 메모리 사용량을 줄이고 연산 속도를 높이기 위함
    with torch.no_grad():
        # input은 이미 1차 임베딩은 되어 있는 것을 가져왔다고 가정한다.
        for idx, (batch_cs, batch_ns, batch_label) in enumerate(dataloader):
            n_steps+=1
            # mfcc embedding encoder
            cs_rep, ns_rep, label = model(x_raw=[batch_cs, batch_ns, batch_label])

            batch_label = torch.LongTensor([torch.argmax(label).item()]).to(device)
            scores = model(x_rep=[cs_rep, ns_rep])
            
#             print(f'eval score : {scores}')
                
            loss = criterion(scores, batch_label)

            eval_loss += loss.item()
            
            print(f'predict : {torch.argmax(scores, axis=1)}')
#             print(f'label  : {batch_label}')
            if torch.argmax(scores, axis=1) == batch_label:
                acc += 1
    if n_steps == 0:
        raise Exception('Eval Metric must have at least one example before it can be computed.')
    results = {
        'eval_accuracy': acc / n_steps,
        'eval_loss': eval_loss / n_steps
    }
    return results
    

In [23]:
import time
import os
# from transformers.optimization import AdamW, get_linear_schedule_with_warmup

def train(model, raw_data, epochs, batch_size, output_dir, lr, device='cpu', betas=(0.9, 0.999), weight_decay: float = 0.01, val_ratio=0.2, fp16=False, fp16_opt_level='O1'):
    # fp16=True면 amp를 통한 mixed preicision training을 한다는 의미
    # 사용 조건 : Volta 이상의 nvidia 그래픽 카드(v100, rtx2080ti, 등)
    if fp16:
        try:
            from apex import amp
        except ImportError:
            raise ImportError("Please install apex from https://www.github.com/nvidia/apex to use fp16 training.")
        model, optimizer = amp.initialize(model, optimizer, opt_level=fp16_opt_level)
    
    def train_batch(data, bsz, shuffle=True):
        cs_rep, ns_rep, labels = data
        length = cs_rep.size()[0]
        
        for i in range(0, length, bsz):
            cs_batch = cs_rep[i:i+bsz]
            ns_batch = ns_rep[i:i+bsz]
            label_batch = labels[i:i+bsz]
            
            if shuffle:
                indexes = torch.randperm(bsz)              
                yield [cs_batch[indexes], ns_batch[indexes], label_batch[indexes]]
            else:
                yield [cs_batch, ns_batch, label_batch]    
    state_save_path = os.path.join(output_dir, '{}_{}_{}_pytorch_model.bin'.format('polyencoder', '64', '0815'))
        
    best_eval_loss = float('inf')
    best_test_loss = float('inf')
    
    init_time = time.time()
    
    cs, ns, labels = raw_data[0].to(device), raw_data[1].to(device), raw_data[2].to(device)

    # 재사용 가능한 임베딩들은 미리 뽑기
    trainset = None
    valset = None

    if val_ratio > 0:
        train_len = int((1-val_ratio)*len(labels))
        trainset = [cs[:train_len], ns[:train_len], labels[:train_len]]
        valset = [cs[train_len:], ns[train_len:], labels[train_len:]]
    else:
        trainset = raw_data

    eval_freq = (len(trainset[2]) / batch_size) // 3
    
    t_total = len(trainset[0])

    model.to(device)
    
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, betas=betas, weight_decay=weight_decay)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
            optimizer, mode='min', factor=0.2, threshold=0.01, patience=5)

    # Set the learning rate of each parameter group using a cosine annealing schedule,
    # https://pytorch.org/docs/1.8.0/optim.html?highlight=cosineannealingwarmrestarts#torch.optim.lr_scheduler.CosineAnnealingWarmRestarts
#     scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=10, T_mult=2)
    
    # stepRL
#     scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=eval_freq, gamma=0.95)

        
    batch_labels = []
    scores = []
    global_step = 0
    #TODO : 나중에 tqdm으로 바꾸기
    for epoch in range(1, epochs+1):
        start_time = time.time()
        train_batches = train_batch(trainset, batch_size)
        
        total_loss = 0.0
        accuracy = 0.0
        local_step = 0
        for idx, (batch_cs, batch_ns, batch_label) in enumerate(train_batches):
            model.train()
            
            # mfcc embedding encoder
            cs_rep, ns_rep, label = model(x_raw=[batch_cs, batch_ns, batch_label])
            
            # label
            batch_label = torch.LongTensor([torch.argmax(batch_label).item()]).to(device)
            
            # dot product
            scores = model(x_rep=[cs_rep, ns_rep])
#             print(f'scores : {scores}')

            # cross entropy
            loss = criterion(scores, batch_label)

            total_loss += loss.item()
            
            predict = torch.argmax(scores, axis=1)
#             print('predict : ',predict.item())
#             print(f'label : {batch_label.item()}\n')
#             if predict == torch.argmax(batch_label).item():
            if predict == batch_label:
                accuracy += 1
#                 print(f'-- correct : {accuracy}\n')
            
            if torch.isnan(loss):
                print(f'scores : {scores}')
                exit()
            
            if fp16:
                with amp.scale_loss(loss, optimizer) as scaled_loss:
                    scaled_loss.backward()
            else:
                loss.backward()

            optimizer.step()
#             scheduler.step()

            # clear gradient
            optimizer.zero_grad()

            
#             model.zero_grad()
            
#             if global_step and global_step % eval_freq == 0:
#                 if valset is not None:
#                     val_batches = train_batch(valset, batch_size, shuffle=False)

#                 eval_results = eval_model(val_batches, criterion)
#                 scheduler.step(eval_results['eval_loss'])

#                 print(f'Global Step {global_step} Eval loss : {eval_results["eval_loss"]}, Eval accuracy : {eval_results["eval_accuracy"]}')
#                 if eval_results["eval_loss"] < best_eval_loss:
#                     best_eval_loss = eval_results['eval_loss']
#                     # save model.state_dict()
#                     torch.save(model.state_dict(), state_save_path)
#                     print(f'[Saving at] {state_save_path}')

            
            global_step += 1
            local_step += 1

            #print(f'batch {idx} | Loss {loss.item()}')
        
        total_loss = total_loss / local_step
        train_accuracy = accuracy / local_step
        print(f'\n[Epoch {epoch}] | train_cost {total_loss} | train_accuracy {train_accuracy} | time {time.time()-start_time}')
        
        if valset is not None:
            val_batches = train_batch(valset, batch_size, shuffle=False)
        eval_results = eval_model(model, val_batches, criterion, device)
        print(f'Eval loss : {eval_results["eval_loss"]}, Eval accuracy : {eval_results["eval_accuracy"]}')
        if eval_results["eval_loss"] < best_eval_loss:
            best_eval_loss = eval_results['eval_loss']
            # save model.state_dict()
            torch.save(model.state_dict(), state_save_path)
#             print(f'[Saving at] {state_save_path}')
        print('---------------------------------\n')
            

In [24]:
# RuntimeError: Expected 4-dimensional input for 4-dimensional weight [256, 1, 3, 3], but got 3-dimensional input of size [320, 40, 2800] instead
# tdata = [now_mfcc_list.to(1), next_mfcc_list.to(1), label_list.to(1)]
tdata = [cs, ns, labels]
# batch_size, C, H, W = 32, 1, 28, 28
# tdata = [torch.randn(batch_size, C, H, W).to(1),torch.randn(batch_size, C, H, W).to(1), torch.randn(batch_size).to(1)]

# Hyperparameters
epochs = 100 # epoch
batch_size=32 # batch size for training
lr=1e-5 # learning rate

# criterion = torch.nn.CrossEntropyLoss()
# optimizer = torch.optim.Adam(model.parameters(), lr=lr)
# scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
#             optimizer, mode='min', factor=0.2, threshold=0.01, patience=5)
device='cuda:1'

train(model=model, 
      raw_data=tdata, 
      epochs=epochs, 
      batch_size=batch_size, 
      output_dir='.', 
      lr=lr,
      device=device,
      val_ratio=0.2, 
      fp16=False, 
      fp16_opt_level='O1')


[Epoch 1] | train_cost 6.365438917279244 | train_accuracy 0.0375 | time 74.18418383598328
predict : tensor([18], device='cuda:1')
label  : tensor([0], device='cuda:1')
predict : tensor([1], device='cuda:1')
label  : tensor([0], device='cuda:1')
predict : tensor([25], device='cuda:1')
label  : tensor([0], device='cuda:1')
predict : tensor([27], device='cuda:1')
label  : tensor([0], device='cuda:1')
predict : tensor([6], device='cuda:1')
label  : tensor([0], device='cuda:1')
predict : tensor([19], device='cuda:1')
label  : tensor([0], device='cuda:1')
predict : tensor([12], device='cuda:1')
label  : tensor([0], device='cuda:1')
predict : tensor([2], device='cuda:1')
label  : tensor([0], device='cuda:1')
predict : tensor([10], device='cuda:1')
label  : tensor([0], device='cuda:1')
predict : tensor([11], device='cuda:1')
label  : tensor([0], device='cuda:1')
predict : tensor([23], device='cuda:1')
label  : tensor([0], device='cuda:1')
predict : tensor([15], device='cuda:1')
label  : tenso

predict : tensor([11], device='cuda:1')
label  : tensor([0], device='cuda:1')
predict : tensor([30], device='cuda:1')
label  : tensor([0], device='cuda:1')
predict : tensor([3], device='cuda:1')
label  : tensor([0], device='cuda:1')
predict : tensor([15], device='cuda:1')
label  : tensor([0], device='cuda:1')
Eval loss : 4.583966469764709, Eval accuracy : 0.0
---------------------------------


[Epoch 6] | train_cost 3.6381503283977508 | train_accuracy 0.075 | time 74.39221358299255
predict : tensor([1], device='cuda:1')
label  : tensor([0], device='cuda:1')
predict : tensor([31], device='cuda:1')
label  : tensor([0], device='cuda:1')
predict : tensor([7], device='cuda:1')
label  : tensor([0], device='cuda:1')
predict : tensor([26], device='cuda:1')
label  : tensor([0], device='cuda:1')
predict : tensor([18], device='cuda:1')
label  : tensor([0], device='cuda:1')
predict : tensor([19], device='cuda:1')
label  : tensor([0], device='cuda:1')
predict : tensor([12], device='cuda:1')
label 

predict : tensor([7], device='cuda:1')
label  : tensor([0], device='cuda:1')
predict : tensor([19], device='cuda:1')
label  : tensor([0], device='cuda:1')
predict : tensor([30], device='cuda:1')
label  : tensor([0], device='cuda:1')
predict : tensor([5], device='cuda:1')
label  : tensor([0], device='cuda:1')
predict : tensor([30], device='cuda:1')
label  : tensor([0], device='cuda:1')
predict : tensor([22], device='cuda:1')
label  : tensor([0], device='cuda:1')
predict : tensor([1], device='cuda:1')
label  : tensor([0], device='cuda:1')
predict : tensor([9], device='cuda:1')
label  : tensor([0], device='cuda:1')
predict : tensor([15], device='cuda:1')
label  : tensor([0], device='cuda:1')
Eval loss : 5.204466366767884, Eval accuracy : 0.0
---------------------------------


[Epoch 11] | train_cost 3.0279651559889316 | train_accuracy 0.175 | time 74.39689564704895
predict : tensor([1], device='cuda:1')
label  : tensor([0], device='cuda:1')
predict : tensor([31], device='cuda:1')
label  

predict : tensor([12], device='cuda:1')
label  : tensor([0], device='cuda:1')
predict : tensor([25], device='cuda:1')
label  : tensor([0], device='cuda:1')
predict : tensor([31], device='cuda:1')
label  : tensor([0], device='cuda:1')
predict : tensor([11], device='cuda:1')
label  : tensor([0], device='cuda:1')
predict : tensor([10], device='cuda:1')
label  : tensor([0], device='cuda:1')
predict : tensor([7], device='cuda:1')
label  : tensor([0], device='cuda:1')
predict : tensor([16], device='cuda:1')
label  : tensor([0], device='cuda:1')
predict : tensor([30], device='cuda:1')
label  : tensor([0], device='cuda:1')
predict : tensor([5], device='cuda:1')
label  : tensor([0], device='cuda:1')
predict : tensor([28], device='cuda:1')
label  : tensor([0], device='cuda:1')
predict : tensor([11], device='cuda:1')
label  : tensor([0], device='cuda:1')
predict : tensor([10], device='cuda:1')
label  : tensor([0], device='cuda:1')
predict : tensor([9], device='cuda:1')
label  : tensor([0], device

predict : tensor([1], device='cuda:1')
label  : tensor([0], device='cuda:1')
predict : tensor([31], device='cuda:1')
label  : tensor([0], device='cuda:1')
predict : tensor([10], device='cuda:1')
label  : tensor([0], device='cuda:1')
predict : tensor([26], device='cuda:1')
label  : tensor([0], device='cuda:1')
predict : tensor([7], device='cuda:1')
label  : tensor([0], device='cuda:1')
predict : tensor([19], device='cuda:1')
label  : tensor([0], device='cuda:1')
predict : tensor([14], device='cuda:1')
label  : tensor([0], device='cuda:1')
predict : tensor([7], device='cuda:1')
label  : tensor([0], device='cuda:1')
predict : tensor([31], device='cuda:1')
label  : tensor([0], device='cuda:1')
predict : tensor([11], device='cuda:1')
label  : tensor([0], device='cuda:1')
predict : tensor([25], device='cuda:1')
label  : tensor([0], device='cuda:1')
predict : tensor([24], device='cuda:1')
label  : tensor([0], device='cuda:1')
predict : tensor([16], device='cuda:1')
label  : tensor([0], device

predict : tensor([6], device='cuda:1')
label  : tensor([0], device='cuda:1')
predict : tensor([9], device='cuda:1')
label  : tensor([0], device='cuda:1')
predict : tensor([15], device='cuda:1')
label  : tensor([0], device='cuda:1')
Eval loss : 6.937251555919647, Eval accuracy : 0.0
---------------------------------


[Epoch 25] | train_cost 2.3176280558109283 | train_accuracy 0.3 | time 74.38817310333252
predict : tensor([16], device='cuda:1')
label  : tensor([0], device='cuda:1')
predict : tensor([15], device='cuda:1')
label  : tensor([0], device='cuda:1')
predict : tensor([31], device='cuda:1')
label  : tensor([0], device='cuda:1')
predict : tensor([26], device='cuda:1')
label  : tensor([0], device='cuda:1')
predict : tensor([7], device='cuda:1')
label  : tensor([0], device='cuda:1')
predict : tensor([19], device='cuda:1')
label  : tensor([0], device='cuda:1')
predict : tensor([12], device='cuda:1')
label  : tensor([0], device='cuda:1')
predict : tensor([6], device='cuda:1')
label  :

predict : tensor([16], device='cuda:1')
label  : tensor([0], device='cuda:1')
predict : tensor([13], device='cuda:1')
label  : tensor([0], device='cuda:1')
predict : tensor([5], device='cuda:1')
label  : tensor([0], device='cuda:1')
predict : tensor([28], device='cuda:1')
label  : tensor([0], device='cuda:1')
predict : tensor([22], device='cuda:1')
label  : tensor([0], device='cuda:1')
predict : tensor([17], device='cuda:1')
label  : tensor([0], device='cuda:1')
predict : tensor([9], device='cuda:1')
label  : tensor([0], device='cuda:1')
predict : tensor([15], device='cuda:1')
label  : tensor([0], device='cuda:1')
Eval loss : 6.9957984447479244, Eval accuracy : 0.0
---------------------------------


[Epoch 30] | train_cost 2.1469969348050655 | train_accuracy 0.3375 | time 74.38450241088867
predict : tensor([1], device='cuda:1')
label  : tensor([0], device='cuda:1')
predict : tensor([31], device='cuda:1')
label  : tensor([0], device='cuda:1')
predict : tensor([10], device='cuda:1')
lab

predict : tensor([6], device='cuda:1')
label  : tensor([0], device='cuda:1')
predict : tensor([18], device='cuda:1')
label  : tensor([0], device='cuda:1')
predict : tensor([11], device='cuda:1')
label  : tensor([0], device='cuda:1')
predict : tensor([25], device='cuda:1')
label  : tensor([0], device='cuda:1')
predict : tensor([24], device='cuda:1')
label  : tensor([0], device='cuda:1')
predict : tensor([16], device='cuda:1')
label  : tensor([0], device='cuda:1')
predict : tensor([13], device='cuda:1')
label  : tensor([0], device='cuda:1')
predict : tensor([5], device='cuda:1')
label  : tensor([0], device='cuda:1')
predict : tensor([28], device='cuda:1')
label  : tensor([0], device='cuda:1')
predict : tensor([22], device='cuda:1')
label  : tensor([0], device='cuda:1')
predict : tensor([1], device='cuda:1')
label  : tensor([0], device='cuda:1')
predict : tensor([9], device='cuda:1')
label  : tensor([0], device='cuda:1')
predict : tensor([15], device='cuda:1')
label  : tensor([0], device=

predict : tensor([31], device='cuda:1')
label  : tensor([0], device='cuda:1')
predict : tensor([10], device='cuda:1')
label  : tensor([0], device='cuda:1')
predict : tensor([26], device='cuda:1')
label  : tensor([0], device='cuda:1')
predict : tensor([7], device='cuda:1')
label  : tensor([0], device='cuda:1')
predict : tensor([3], device='cuda:1')
label  : tensor([0], device='cuda:1')
predict : tensor([14], device='cuda:1')
label  : tensor([0], device='cuda:1')
predict : tensor([25], device='cuda:1')
label  : tensor([0], device='cuda:1')
predict : tensor([10], device='cuda:1')
label  : tensor([0], device='cuda:1')
predict : tensor([11], device='cuda:1')
label  : tensor([0], device='cuda:1')
predict : tensor([25], device='cuda:1')
label  : tensor([0], device='cuda:1')
predict : tensor([24], device='cuda:1')
label  : tensor([0], device='cuda:1')
predict : tensor([16], device='cuda:1')
label  : tensor([0], device='cuda:1')
predict : tensor([13], device='cuda:1')
label  : tensor([0], devic

KeyboardInterrupt: 